In [1]:
from datasets import load_dataset
dataset = load_dataset("Cartinoe5930/raw_text_synthetic_dataset_50k", split = "train")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 21900/21900 [00:00<00:00, 96257.80 examples/s] 


# 역색인

In [3]:
from kiwipiepy import Kiwi
from collections import defaultdict
import math

In [4]:
class InvertedIndex:
    def __init__(self):
        self.reset()

    def reset(self):
        self.index = defaultdict(dict)
        self.kiwi = Kiwi()
        self.document_lengths = {}
        self.total_documents = 0
        self.average_document_length = 0
        self.documents = {}

    def tokenize(self, text):
        return [token.form for token in self.kiwi.tokenize(text)]

    def add_document(self, doc_id, question, answer):
        tokens = self.tokenize(question)
        self.document_lengths[doc_id] = len(tokens)
        self.total_documents += 1
        self.documents[doc_id] = {'question': question, 'answer': answer}

        for token in set(tokens):
            if doc_id not in self.index[token]:
                self.index[token][doc_id] = 0
            self.index[token][doc_id] += tokens.count(token)

        self.average_document_length = sum(self.document_lengths.values()) / self.total_documents

    def calculate_bm25_score(self, query_tokens, doc_id):
        k1 = 1.5
        b = 0.75
        score = 0

        for token in query_tokens:
            if token not in self.index or doc_id not in self.index[token]:
                continue

            tf = self.index[token][doc_id]
            df = len(self.index[token])
            idf = math.log((self.total_documents - df + 0.5) / (df + 0.5) + 1)

            numerator = tf * (k1 + 1)
            denominator = tf + k1 * (1 - b + b * self.document_lengths[doc_id] / self.average_document_length)
            score += idf * numerator / denominator

        return score

    def search(self, query, k=5):
        query_tokens = self.tokenize(query)
        scores = defaultdict(float)

        for token in query_tokens:
            if token in self.index:
                for doc_id in self.index[token]:
                    scores[doc_id] += self.calculate_bm25_score(query_tokens, doc_id)

        top_k = sorted(scores.items(), key=lambda x: x[1], reverse = True)[:k]
        return [(doc_id, score, self.documents[doc_id]) for doc_id, score in top_k]

In [5]:
# add docs
from tqdm import tqdm

index = InvertedIndex()
for idx, data in enumerate(tqdm(dataset)):
    question = data['question']
    answer = data['response']
    index.add_document(idx, question, answer)

100%|██████████| 21900/21900 [00:47<00:00, 456.81it/s]


# 유사 문제 찾기

In [48]:
# 2000
question = dataset[100]['question']
answer = dataset[100]['response']

print(question)
print(answer)

similar_questions = index.search(question, k=15)
for quest in similar_questions:
    print(quest)

2019년에 번역이 진행된 여러 금융 AI 가이드라인 중에서, AI 시스템이 공정하고 투명하며 설명 가능한 이유가 무엇인지, 또한 이러한 시스템을 개발하기 위해 금융 서비스 조직이 어떤 절차를 마련해야 하는지를 설명하는 데 필요한 여러 단계의 추론은 무엇입니까?
AI 시스템이 공정하고 투명하며 설명 가능해야 하는 이유는 다음과 같습니다:

1. **신뢰 구축**: 금융 서비스 분야에서 고객과의 신뢰는 핵심입니다. 공정하고 투명한 AI 시스템은 고객이 시스템의 결정 과정을 이해하고 신뢰할 수 있도록 도와줍니다.

2. **편향 제거**: AI 모델이 훈련 데이터의 편향을 반영하지 않도록 하는 것이 중요합니다. 공정성을 통해 특정 그룹에 대한 시스템의 차별을 방지할 수 있습니다.

3. **규제 준수**: 많은 국가에서는 금융 서비스에 대한 규제가 강화되고 있으며, 이러한 규제를 준수하기 위해서는 AI 시스템의 투명성과 설명 가능성이 필수적입니다.

4. **결정 과정의 이해**: 설명 가능한 AI는 사용자가 시스템의 결정을 더 잘 이해할 수 있도록 하여, 잘못된 결정에 대한 피드백과 개선이 용이합니다.

금융 서비스 조직이 공정하고 투명하며 설명 가능한 AI 시스템을 개발하기 위해 반드시 숙지해야 할 절차의 여러 단계는 다음과 같습니다:

1. **윤리적 프레임워크 정의**: 조직의 AI 시스템이 따라야 할 윤리적 기준을 설정하고, 공정성, 투명성, 설명 가능성에 대한 명확한 가이드라인을 마련합니다.

2. **데이터 관리 및 품질 보증**: 공정성을 위해 다양하고 포괄적인 데이터셋을 수집하고, 데이터의 편향을 평가하며, 필요한 경우 데이터를 정제해야 합니다.

3. **모델 개발 프로세스**: AI 모델 개발 시 공정성과 투명성을 보장할 수 있는 알고리즘과 기법을 적용합니다. 모델의 결정 과정을 설명할 수 있는 방법론을 사용해야 합니다.

4. **테스트 및 검증**: AI 시스템의 공정성, 정확성 및 설명 가능성을 검증하기 위한 테스트 환경을 마련하고, 다양

# 비슷한 문제들을 참고해서 만들게 하자

In [12]:
import openai
from dotenv import load_dotenv
import os

dotenv_path = os.path.join(os.getcwd(), '.env')
load_dotenv(dotenv_path)

openai_key = os.getenv('OPENAI_TEAM_API_KEY')
client = openai.OpenAI(api_key=openai_key)

In [13]:
system_prompt = """
You are given two pairs of reference questions and reference answers.
Your role is a questioner who make a new question.
When making your questions, consider the following.
1. New Question must require choices such as 'Which is right', 'Which is not right', 'Which is most appropriate', and 'Which is not most appropriate'.
2. You have to make 5 choices, 1 answer choice and 4 wrong choices.
3. The choices must be generated in association with one of several keywords in the reference question and answer.
4. The wrong answer and the right answer are confused, but the wrong answer must be a clear wrong answer.
5. The choices does not deviate from the subject of the problem, but it must be different.
6. The choices sentence must be similar in length.
7. If a person is a financial expert, the person can solve the problem, but if the person is a beginner in financial knowledge, please make the problem with a difficulty that the person cannot solve because it is difficult.
8. Please don't create a problem that can be solved by reading other than the problem.
please write in Korean and you must write the answer on the last line.
"""

user_prompt ="""
### Reference
### Question 1: {}
### Answer 1: {}

### Question 2: {}
### Answer 2: {}

### New Question : 

"""


# 모든 데이터 뽑기

In [14]:
len(dataset)

21900

In [15]:
import random
from tqdm import tqdm

new_questions = []
for idx, data in enumerate(tqdm(dataset, total=len(dataset))):
    question = data['question']
    answer = data['response']
    system_msg = {'role':'system', 'content': system_prompt}

    similar_questions = index.search(question, k=2)

    user_msg = {'role':'user', 'content': user_prompt.format(
        similar_questions[0][2]['question'],
        similar_questions[0][2]['answer'],
        similar_questions[1][2]['question'],
        similar_questions[1][2]['answer'])}

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages = [system_msg, user_msg],
    )

    result = response.choices[0].message.content
    new_questions.append(result)
    
    if idx % 100 == 0:
        print(result)
        print("*"*100)


  0%|          | 1/21900 [00:27<165:04:34, 27.14s/it]

AI 시스템이 결정하는 신용 점수가 공정하고 신뢰할 수 있도록 설계하기 위한 방안 중에서 가장 적절하지 않은 방법은 무엇인가요? 선택지를 다음과 같이 제시합니다:

1. **고품질의 다양한 데이터를 수집하는 것**  
2. **연령이나 성별에 따라 데이터를 제한하는 것**  
3. **윤리적 원칙을 수립하여 데이터 사용을 규제하는 것**  
4. **AI 시스템의 결정 과정을 외부에 공개하지 않는 것**  
5. **지속적인 데이터 검증 및 업데이트를 수행하는 것**  

정답은: 2. **연령이나 성별에 따라 데이터를 제한하는 것**
****************************************************************************************************


  0%|          | 101/21900 [31:18<145:06:25, 23.96s/it]

2019년 발표된 AI 시스템의 투명성, 공정성, 설명 가능성에 대한 가이드라인에 기반하여 금융 서비스 조직이 따라야 할 절차 중에서 무엇이 가장 적절한지 선택해 보세요. 

1. 모든 직원에게 AI 시스템의 작동 방식을 기밀로 유지하게 한다.
2. 고객의 피드백을 무시하고 내부 개발만 실시한다.
3. 알고리즘의 결정 과정에 대해 고객에게 투명하게 공개한다.
4. 데이터 수집 과정에서 특정 집단만을 대상으로 한다.
5. 회의 후에도 절차를 지속적으로 업데이트하지 않는다.

정답: 3. 알고리즘의 결정 과정에 대해 고객에게 투명하게 공개한다.
****************************************************************************************************


  1%|          | 201/21900 [58:33<126:27:23, 20.98s/it]

...

AI 시스템이 금융 서비스에서 편견 문제를 해결하기 위한 노력이 필요하다고 할 때, 어떤 조치가 가장 적절한지 선택해 보세요. 다음 중 어떤 것이 올바른 조치가 아닐까요?

1. 데이터 세트를 철저히 검토하고, 사회적 편견을 제거하기 위한 노력이 필요하다.
2. AI 모델의 학습 데이터에 과거의 차별적 사례를 그대로 포함시키는 것이 중요하다.
3. 지속적인 모니터링을 통해 편향 요소를 실시간으로 검토하고 개선해야 한다.
4. 모든 고객으로부터 피드백을 수집하여 시스템 개선에 반영해야 한다.
5. 편견 감지를 위한 알고리즘을 개발하고 이를 AI 모델에 통합해야 한다.

정답: 2. AI 모델의 학습 데이터에 과거의 차별적 사례를 그대로 포함시키는 것이 중요하다.
****************************************************************************************************


  2%|▏         | 401/21900 [1:59:29<82:03:01, 13.74s/it] 

중국과 인도가 금융 포용성을 높이기 위해 시행한 조치 중에서 어느 것이 올바른지 판단해 보세요. 다음 중 올바르지 않은 것을 고르세요.

1. 디지털 금융 서비스의 확대로 재정 서비스 접근성이 증가했다.
2. 정부가 소규모 기업에 대한 대출을 축소하는 정책을 추진했다.
3. 마이크로파이낸스를 통해 저소득층의 사업 기회를 증대시켰다.
4. 금융 교육 프로그램을 통해 대중의 재정 관리 능력을 높였다.
5. 소득 및 지출 기록의 디지털화를 통해 신용을 구축할 수 있는 기회를 제공했다.

정답: 2. 정부가 소규모 기업에 대한 대출을 축소하는 정책을 추진했다.
****************************************************************************************************


  2%|▏         | 415/21900 [2:02:52<106:01:35, 17.77s/it]


KeyboardInterrupt: 